In [2]:
import os
from docx import Document
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
api_key = os.getenv("API_KEY")
client = OpenAI(api_key=api_key)

# Folder with all the summaries of the doctor
folder_path = 'summaries'

# Step 1: Extract Text from Each Document
documents_texts = []
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path, filename)
        print(f"Reading file: {filename}")

        # Reading .doc file
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs]).strip()
        documents_texts.append(text)

# Step 2: Comparative Analysis for Template Updates
comparative_prompt = "Compare these medical summary documents and identify any template updates or structural changes, such as the addition of information about the patient or the appointment that are not consistently present in all documents. If the majority of the documents have such a new inclusion, consider it as a template update. State then the structure of the summaries with the different inclusions: " + " /// ".join(documents_texts)
# comparative_prompt = "Compare these medical summary documents and identify any major template updates or structural changes, such as the addition of new sections like mood or other elements that are not consistently present in all documents: " + " /// ".join(documents_texts)

# Call for comparative analysis
comparative_response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": comparative_prompt}
    ],
    model="gpt-4"
)

# Extracting and Printing comparative analysis results
if comparative_response.choices and len(comparative_response.choices) > 0:
    comparative_analysis = comparative_response.choices[0].message.content  # Corrected access
    print("Comparative Analysis:\n", comparative_analysis)
else:
    print("No comparative analysis result")

Reading file: summ_1.docx
Reading file: summ_2.docx
Reading file: summ_3.docx
Comparative Analysis:
 All three medical summaries have similar structure and content with slight variations due to the differing nature of the patients' concerns and their personal information.

Here's the general structure they follow:

1. Name of the patient and their emotional state during appointment time.
2. Description of the chronic problem and its origins.
3. Frequency and nature of the problem's episodes.
4. Patients' previous treatment regimen and its cessation upon condition improvement.
5. Description of the initial effectiveness and the advantages of the earlier treatment.
6. Reasons for stopping the earlier treatment.
7. Suggestion of a home remedy for temporary relief.
8. Proposition of alternative therapies and their potential benefits.
9. Patients' reaction to the proposed treatment.
10. Discussion of insurance-related issues and possible options.

However, only the summaries of Ms. Smith an

In [5]:
# Load the transcript
transcript = """
It's 11:33 AM. Hi, Mr. Jones. 
How are you?
Good, Dr.Pitt., a bit nervous, lately I've been anxious.
Nice to see you.
Nice to see you again.
What brings you back tonight?
Yeah, well, ever since I got hurt on the job three years ago, it's something that just keeps coming back.
It'll be fine for a while, and then I'll bend down or move in a weird way, and then, boom, it'll just go out again.
Unfortunately, that didn't happen.
And I do have quite a few patients who get reoccurring episodes of back pain.
Have you been keeping up with the therapy that we had you on before?
Which pills?
Actually, I was talking about the physical therapy that we had you doing.
The pills are only meant for short term because they don't actually prevent the back pain from coming back.
See, yeah, once my back started feeling better, I was happy about it, back to work.
So, and I could use the money.
Do you think the physical therapy was helping?
Yeah, it was slow going at first.
I see.
Physical therapy is a bit slower than medications, but the point is to build up the core muscles in your back and your abdomen.
Physical therapy is also less invasive than medications.
So that's why we had you doing the therapy.
But you mentioned that cost was getting to be a real issue for you.
Can you tell me more about that?
Well, the insurance I had only covered a certain number of sessions, and then they moved my therapy office because they were trying to work out my schedule of work.
But that was really far away, and then I had to deal with parking, and it just started to get really expensive.
Got it.
I understand.
So for now I'd like you to try using a heating pad for your back pain.
So that should help in the short term.
Our goal is to get your back pain under better control without creating additional problems for you like cost.
Let's talk about some different options and the pros and cons of each.
So the physical therapy is actually really good for your back pain, but there are other things we can be doing to help.
Yes, I definitely don't need to lose any more time at work and just lie around the house all day.
Okay, well there are some alternative therapies like yoga or tai chi classes or meditation therapies that might be able to help, generic medication.
And then there are also resources for people to look up the cheapest cost of their medications.
But for now, I'd like to stick with the non-prescription medications.
And if we can have you go to yoga or Tai Chi classes, like I mentioned, that could alleviate the need for ordering prescriptions.
Yeah, that sounds good.
Okay, great, great.
Are there any other costs that are a problem for you in your care?
Well, my insurance isn't going down, but that seems to be the case for everybody that I talk to, but I should be able to make it work, yeah.
Unfortunately, that is an issue for a lot of people, but I would encourage you during open season to look at your different insurance options to see which plan is more cost effective for you.
""" 



In [9]:
# Step 1: Generate a summary template based on comparative_analysis
template_generation_prompt = f"Create a summary for a medical transcript maintaining the sections described in this template updates: {comparative_analysis}. Avoid any type of speculation. Directly create a summary for the following transcript without any introductory text:  "
template_response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": template_generation_prompt}
    ],
    model="gpt-4"
)

template = template_response.choices[0].message.content if template_response.choices else "Template generation failed"


# Step 2: Use the generated template to summarize the transcript
summary_prompt = f"Using this template: {template}\n\nCreate a summary for the following transcript: {transcript}"
summary_response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": summary_prompt}
    ],
    model="gpt-4"
)
summary = summary_response.choices[0].message.content if summary_response.choices else "Summary generation failed"

# Print the summary
print(f"Generated Summary:\n{summary}")

Generated Summary:
Patient: Mr. Jones
Emotional state: Expressed nervousness and anxiety during the visit.

Chronic Problem: Mr. Jones has ongoing back pain, an issue that started three years ago due to a work-related injury. The discomfort comes and goes, often triggered by certain movements.

Frequency and nature of the problem's episodes: The back pain recurs after periods of improvement, particularly after Mr. Jones bends down or moves in an unexpected manner.

Previous Treatment: Physical therapy and short-term medication were previously recommended to manage the back pain.

Effectiveness of earlier treatment: Mr. Jones felt the physical therapy was slowly but helping. However, he stopped it once his back started feeling better and he could return to work.

Reason for stopping treatment: Following the initial coverage, Mr. Jones' insurance only covered a limited number of physical therapy sessions. The venue for the session was also moved, adding to the overall expense due to trav